<a href="https://colab.research.google.com/github/samiha-mahin/Data-Analysis/blob/main/FE_Sklearn_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install -U scikit-learn

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder , MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv('/content/titanic.csv')

In [5]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


**Drop Unnecessary Columns**


 These columns are likely:

* PassengerId: Just an identifier, not useful for prediction.

* Name: Might contain useful info, but needs complex text processing.

* Ticket: No consistent structure.

* Cabin: Too many missing values (NaNs).

* So, to simplify, you drop them.







In [6]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                 random_state=30)

**✅ Example: X_train.head()**

| Pclass | Sex    | Age  | SibSp | Parch | Fare    | Embarked |
| ------ | ------ | ---- | ----- | ----- | ------- | -------- |
| 3      | male   | 22.0 | 1     | 0     | 7.2500  | S        |
| 1      | female | 38.0 | 1     | 0     | 71.2833 | C        |
| 3      | female | 26.0 | 0     | 0     | 7.9250  | S        |
| 1      | female | 35.0 | 1     | 0     | 53.1000 | S        |
| 3      | male   | 35.0 | 0     | 0     | 8.0500  | S        |


**✅ Example: y_train.head()**

| Survived |
| -------- |
| 0        |
| 1        |
| 1        |
| 1        |
| 0        |

Each value corresponds to the survival status of the matching row in X_train.

**✅ Example: X_test.head()**

| Pclass | Sex    | Age  | SibSp | Parch | Fare   | Embarked |
| ------ | ------ | ---- | ----- | ----- | ------ | -------- |
| 3      | male   | 28.0 | 0     | 0     | 8.4583 | Q        |
| 2      | female | 19.0 | 0     | 0     | 13.000 | S        |
| 1      | male   | 45.0 | 1     | 1     | 83.475 | S        |
| 3      | male   | NaN  | 0     | 0     | 7.2250 | C        |
| 2      | female | 30.0 | 0     | 0     | 12.350 | Q        |

**✅ Example: y_test.head()**

| Survived |
| -------- |
| 0        |
| 1        |
| 1        |
| 0        |
| 1        |



In [8]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
199,2,female,24.0,0,0,13.0000,S
703,3,male,25.0,0,0,7.7417,Q


In [9]:
y_train.sample(5)

,Survived
223,0
46,0
444,1
672,0
126,0


In [10]:
#imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), #Name of this transformation: impute_age , Column index 2 → Age column
    ('impute_embarked',SimpleImputer(strategy ='most_frequent'),[6]) #this fills missing embarkation values with the most common (mode) value.
],remainder='passthrough')

In [11]:
# one hot encoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(handle_unknown='ignore'),[1,6]) #Column 1 → Sex (male, female),Column 6 → Embarked (C, Q, S)
],remainder='drop')



### 💡 What is `MaxAbsScaler`?

`MaxAbsScaler` is a data normalization technique from scikit-learn. It **scales features by their maximum absolute value** so that **each feature will be in the range \[-1, 1]**, but it **does not shift/center the data (no mean subtraction).**

---

### 🧮 How it works:

For each feature (column), it divides all values by the **maximum absolute value** of that column:

$$
X_{\text{scaled}} = \frac{X}{\max(|X|)}
$$

Example:

* Original: `[-10, 0, 5, 10]`
* Max Abs: `10`
* Scaled: `[-1.0, 0.0, 0.5, 1.0]`

---

### ✅ Why & When to Use:

* Works well with **sparse data** (like after OneHotEncoding)
* **Preserves sparsity** (doesn’t convert to dense arrays)
* Good when **data is already centered at 0** or you **don’t want to shift it**
* Safe with **non-negative data** too

---

### ⚠️ Compared to `MinMaxScaler`:

| Scaler         | Output Range         | Supports Sparse Input | Centers Data |
| -------------- | -------------------- | --------------------- | ------------ |
| `MinMaxScaler` | `[0, 1]` (or custom) | ❌ No                  | ❌ No         |
| `MaxAbsScaler` | `[-1, 1]`            | ✅ Yes                 | ❌ No         |

---


**"Apply MaxAbsScaler to the first 10 columns."**

After previous transformations (like one-hot encoding), your dataset might have many more columns than before — especially because Sex and Embarked were turned into multiple binary columns.Takes all numeric and encoded columns (up to column index 10). Applies MaxAbsScaler to scale each feature independently between -1 and 1.

In [12]:
from sklearn.preprocessing import MaxAbsScaler

trf3 = ColumnTransformer([
    ('scale', MaxAbsScaler(), slice(0, 10))
])


In [13]:
# Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [14]:
# train the model
trf5 = DecisionTreeClassifier()

# **Create Pipeline**

In [15]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# **Pipeline Vs make_pipeline**
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [16]:
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [17]:
# train
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MaxAbsScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x78bab7175bc0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [18]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore'),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MaxAbsScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x78bab7175bc0>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [19]:
from sklearn import set_config
set_config(display='diagram')

In [20]:
# Predict
y_pred = pipe.predict(X_test)

In [21]:
y_pred

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6033519553072626

# Cross Validation Using Pipeline

**Cross-validation** is a technique used to evaluate a machine learning model's performance more reliably by splitting the dataset into multiple parts (called folds) and training/testing the model multiple times.

**cross_val_score:**

* cv=5 means: split the data into 5 parts (called folds).

* In each run:

   * Use 4 parts to train the model.

   * Use the remaining 1 part to test the model.

   * This is done 5 times, changing the test part each time.

**Output:**

You get 5 scores (one for each fold).

The .mean() gives the average accuracy, which is more reliable than just one train_test_split().

In [23]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6446666010046291)

# GridSearch using Pipeline

GridSearchCV helps you find the best combination of hyperparameters for a machine learning model. It tries all possible combinations from a grid you define and tells you which gives the best performance.

In [30]:
# gridsearchcv
params = {
    'decisiontreeclassifier__max_depth':[1,2,3,4,5,None]
}
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MaxAbsScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x78bab7175bc0>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [31]:
grid.best_score_

np.float64(0.6446666010046291)

In [32]:
grid.best_params_

{'decisiontreeclassifier__max_depth': 1}

# Exporting the Pipeline

In [34]:
# export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

# Predict Using Pipeline

In [35]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [36]:
# Assume user input
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [37]:
pipe.predict(test_input2)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])